In [ ]:
# Because MobileDet is based out of TensorFlow 1
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

!git clone --quiet https://github.com/tensorflow/models.git

TensorFlow 1.x selected.
1.15.2


In [ ]:
# Install the TF Object Detection API (TFOD)
%%bash
pip install --upgrade pip
cd models/research
# Compile protos.
protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
cp object_detection/packages/tf1/setup.py .
python -m pip install --use-feature=2020-resolver .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1608641 sha256=2390f8dee3147854ff25482daae77299994ceffcb0e89011bb650b874432a49a
  Stored in directory: /tmp/pip-ephem-wheel-cache-4c5f2cck/wheels/16/63/fd/1293066bf448f757979af1b45c59b0e33b46f12d4454d12fde
Successfully built object-detection


# Create tflite graph

Normal model:

In [ ]:
checkpoint_link = 'http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_cpu_320x320_coco_2020_05_19.tar.gz'
folder_name = 'ssdlite_mobiledet_cpu_320x320_coco_2020_05_19'

!rm -rf folder_name
!wget -q $checkpoint_link -O checkpoints.tar.gz
!tar -xvf checkpoints.tar.gz
!rm checkpoints.tar.gz

ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/
ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/model.ckpt-400000.data-00000-of-00001
ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/model.ckpt-400000.index
ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/model.ckpt-400000.meta
ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/tflite_graph.pbtxt
ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/tflite_graph.pb
ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/pipeline.config
ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/model.tflite


In [ ]:
#Export a TFLite compatible model graph
!python /content/models/research/object_detection/export_inference_graph.py \
    --pipeline_config_path=$folder_name/pipeline.config \
    --trained_checkpoint_prefix=$folder_name/model.ckpt-400000 \
    --output_directory=$folder_name \
    --add_postprocessing_op=true
print("======Graph generated========")
!ls -lh $folder_name/*.pb

Instructions for updating:
Please use `layer.__call__` method instead.
W1125 16:34:28.454646 139755282466688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1125 16:34:32.321838 139755282466688 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1125 16:34:32.418491 139755282466688 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1125 16:34:32.521054 139755282466688 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

QAT model:

In [ ]:
checkpoint_link = 'http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz'
folder_name = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19'

!rm -rf folder_name
!wget -q $checkpoint_link -O checkpoints.tar.gz
!tar -xvf checkpoints.tar.gz
!rm checkpoints.tar.gz

#Export a TFLite compatible model graph
precision = 'uint8'

!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$folder_name/$precision/pipeline.config \
    --trained_checkpoint_prefix=$folder_name/$precision/model.ckpt-400000 \
    --output_directory=$folder_name/$precision \
    --add_postprocessing_op=true
print("======Graph generated========")
!ls -lh $folder_name/$precision/*.pb

ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.data-00000-of-00001
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.index
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.meta
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/tflite_graph.pbtxt
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/tflite_graph.pb
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/pipeline.config
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.tflite
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/uint8/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/uint8/model.ckpt-400000.data-00000-of-00001
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/uint8/model.ckpt-400000.index
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/uint8/model.ckpt-400000.meta
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19

# Quantization

In [ ]:
#Initial setup
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

'2.3.0'

Normal model:

In [ ]:
#Select model to quantize
import os

folder_name = 'ssdlite_mobiledet_cpu_320x320_coco_2020_05_19'
model_to_be_quantized = folder_name + '/tflite_graph.pb'
if os.path.getsize(model_to_be_quantized) > 0:
    print(f"We are quantizing: {model_to_be_quantized}")
else:
    print("Model not found!")

We are quantizing: ssdlite_mobiledet_cpu_320x320_coco_2020_05_19/tflite_graph.pb


In [ ]:
# Representative dataset:
!wget -q https://github.com/sayakpaul/Adventures-in-TensorFlow-Lite/releases/download/v0.9.0/train_samples_coco.tar.gz
!tar -xvf train_samples_coco.tar.gz
!rm train_samples_coco.tar.gz

rep_ds = tf.data.Dataset.list_files("train_samples/*.jpg")
HEIGHT, WIDTH = 320, 320

def representative_dataset_gen():
    for image_path in rep_ds:
        img = tf.io.read_file(image_path)
        img = tf.io.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        resized_img = tf.image.resize(img, (HEIGHT, WIDTH))
        resized_img = resized_img[tf.newaxis, :]
        yield [resized_img]

In [ ]:
#Quantize and convert
quant_targets = ["dr", "fp16", "int8_fallback", "normal"]

for quantization_strategy in quant_targets:
  converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
      graph_def_file=model_to_be_quantized, 
      input_arrays=['normalized_input_image_tensor'],
      output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'],
      input_shapes={'normalized_input_image_tensor': [1, 320, 320, 3]}
  )
  converter.allow_custom_ops = True

  if quantization_strategy=="dr":
      converter.optimizations = [tf.lite.Optimize.DEFAULT]
  elif quantization_strategy=="fp16":
      converter.target_spec.supported_types = [tf.float16]
      converter.optimizations = [tf.lite.Optimize.DEFAULT]
  elif quantization_strategy=="int8_fallback":  # Modified for the tf website version... https://www.tensorflow.org/lite/performance/post_training_integer_quant
      converter.optimizations = [tf.lite.Optimize.DEFAULT]
      converter.representative_dataset = representative_dataset_gen
      converter.quantized_input_stats = {"normalized_input_image_tensor": (128, 128)}
  else:
      pass
  tflite_model = converter.convert()

  tflite_filename = 'mobiledet' + "_" + quantization_strategy + ".tflite"
  open(tflite_filename, 'wb').write(tflite_model)
  print(f"TFLite model generated with {quantization_strategy}")


!ls -lh $tflite_filename

TFLite model generated with dr
TFLite model generated with fp16
TFLite model generated with int8_fallback
TFLite model generated with normal
-rw-r--r-- 1 root root 16M Nov 25 16:38 mobiledet_normal.tflite


QAT model:

In [ ]:
#Select model to quantize
import os

model_checkpoint_name = "ssd_mobiledet_edgetpu_coco_uint8"
model_to_be_quantized = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/uint8/tflite_graph.pb'
if os.path.getsize(model_to_be_quantized) > 0:
    print(f"We are quantizing: {model_checkpoint_name}")
else:
    print("Model not found!")

We are quantizing: ssd_mobiledet_edgetpu_coco_uint8


In [ ]:
#Quantize and serialize (`uint8` are WIP)
original_precision = "uint8"
quantization_strategy = "int8"

converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file=model_to_be_quantized, 
    input_arrays=['normalized_input_image_tensor'],
    output_arrays=['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'],
    input_shapes={'normalized_input_image_tensor': [1, 320, 320, 3]}
)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # ADDED BY ME!!!
converter.allow_custom_ops = True

# https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692063937
converter.inference_input_type = tf.uint8
converter.quantized_input_stats = {"normalized_input_image_tensor": (128, 128)}
# No additional optimization: https://github.com/ml-gde/e2e-tflite-tutorials/issues/21#issuecomment-692545579

tflite_model = converter.convert()

tflite_filename = 'mobiledet' + "_" + quantization_strategy + ".tflite"
open(tflite_filename, 'wb').write(tflite_model)
print(f"TFLite model generated with {quantization_strategy}")
!ls -lh $tflite_filename

TFLite model generated with int8
-rw-r--r-- 1 root root 4.2M Nov 25 16:41 mobiledet_int8.tflite


In [ ]:
#Install the latest edge TPU compiler
%%bash
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
sudo apt-get update
sudo apt-get install edgetpu-compiler

In [ ]:
#Compile the file
!edgetpu_compiler -s /content/mobiledet_int8.tflite

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 1498 ms.

Input model: /content/mobiledet_int8.tflite
Input size: 4.13MiB
Output model: mobiledet_int8_edgetpu.tflite
Output size: 5.15MiB
On-chip memory used for caching model parameters: 4.89MiB
On-chip memory remaining for caching model parameters: 2.74MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 137
Operation log: mobiledet_int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 134
Number of operations that will run on CPU: 3

Operator                       Count      Status

ADD                            15         Mapped to Edge

In [ ]:
#Download models
from google.colab import files
import glob
for file in glob.glob("./*.tflite"):
  files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>